In [2]:
!pip install openai itables Wikipedia-API  -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import wikipediaapi
import pandas as pd

# 위키피디아 API 객체 생성 (한국어 버전)
wiki_wiki = wikipediaapi.Wikipedia('Hansoldeco', 'en')

# 크롤링할 카테고리 목록
categories = [
    "Category:Architecture",
    "Category:Environmental_engineering",
    "Category:Structural_engineering",
    "Category:Design",
    "Category:Civil_engineering",
    "Category:Interior_design"
]

# 결과를 저장할 데이터프레임 초기화
df = pd.DataFrame(columns=["Category", "Page Title", "Summary"])

def get_categorymembers(categorymembers, level=0, max_level=1):
    for c in categorymembers.values():
        # 페이지가 카테고리인 경우 재귀적으로 탐색
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            get_categorymembers(c.categorymembers, level=level+1, max_level=max_level)
        else:
            if c.ns == wikipediaapi.Namespace.MAIN:
                page = wiki_wiki.page(c.title)
                summary = page.summary[0:500]  # 요약 내용의 첫 500자만 추출
                df.loc[len(df)] = [c.title, page.title, summary]

# 각 카테고리에 대해 페이지 및 서브 카테고리 탐색
for category_name in categories:
    cat = wiki_wiki.page(category_name)
    print(f"Processing {category_name}...")
    get_categorymembers(cat.categorymembers)

# 결과를 CSV와 JSON 파일로 저장
df.to_csv('/content/drive/MyDrive/DACON_Hansoldeco2/wiki_data.csv', index=False)
df.to_json('/content/drive/MyDrive/DACON_Hansoldeco2/wiki_data.json', orient="records", force_ascii=False)

print("Data extraction and saving completed.")

Processing Category:Architecture...
Processing Category:Environmental_engineering...
Processing Category:Structural_engineering...
Processing Category:Design...
Processing Category:Civil_engineering...
Processing Category:Interior_design...
Data extraction and saving completed.


In [13]:
import itables

table = itables.show(df, table_options=dict(pageLength=10))
table

In [4]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 11.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=caefc2d46a5e02a6c387c82e7a141984acb912dcaddf3f644062c026f1eaf46a
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.

In [8]:
!pip install httpx

In [12]:
from google.cloud import translate_v2 as translate
import pandas as pd

# Google Cloud Translation 클라이언트 설정
translate_client = translate.Client()

# CSV 파일 로드
file_path = '/content/drive/MyDrive/DACON_Hansoldeco2/wiki_data.csv'
df = pd.read_csv(file_path)

def translate_text(text, target='ko'):
    if text:
        translation = translate_client.translate(text, target_language=target)
        return translation['translatedText']
    return None

for column in df.columns:
    df[column] = df[column].apply(lambda x: translate_text(x))

# 번역된 데이터를 새로운 CSV 파일로 저장
translated_file_path = '/content/drive/MyDrive/DACON_Hansoldeco2/wiki_data_translated.csv'
df.to_csv(translated_file_path, index=False)

print("Translation completed and saved to:", translated_file_path)

RefreshError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7db2912f9b70>)

In [ ]:
# 결과를 CSV와 JSON 파일로 저장
df.to_csv('/content/drive/MyDrive/DACON_Hansoldeco2/wiki_data.csv', index=False)
df.to_json('/content/drive/MyDrive/DACON_Hansoldeco2/wiki_data.json', orient="records", force_ascii=False)